In [1]:
!pip install langchain
!pip install boto3
!pip install langchain-community
!pip install langchain-aws
# !pip install sentence_transformers
!pip install gpt4all
# !pip install faiss-gpu
!pip install langchain-qdrant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.

In [2]:
!python -c "from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings; embedding_model = GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf')"

Downloading: 100% 45.9M/45.9M [00:01<00:00, 25.7MiB/s]
Verifying: 100% 45.9M/45.9M [00:00<00:00, 561MiB/s]
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [4]:
# !pip install aws configure
# !pip install awscli

# !aws configure

In [13]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_aws.llms.bedrock import Bedrock, BedrockLLM
# from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_qdrant import QdrantVectorStore

from google.colab import userdata

In [6]:
def get_bedrock_runtime(region_name, *args, **kwargs):
    return boto3.client(service_name='bedrock-runtime', region_name=region_name, *args, **kwargs)


def get_langchain_bedrock_llm(model_id, client, *args, **kwargs):
    # return Bedrock(model_id=model_id, client=client, *args, **kwargs)
    return BedrockLLM(model_id=model_id, client=client, *args, **kwargs)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [10]:
vectorstoredir = '/content/drive/MyDrive/Apologetics Vectorstore'
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
# embed_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
qdrant_url = userdata.get('APOLOGETICS_QDRANT_URL')
qdrant_api_key = userdata.get('LANGSMITH_API_KEY')
llm_name = 'mistral.mixtral-8x7b-instruct-v0:1'

In [11]:
aws_access_token = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_ACCESS_TOKEN')
aws_secret_key = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_SECRET_ACCESS_KEY')

config = {
      "max_tokens": 4096,
      "temperature": 0.7,
      "top_p": 0.8
    }

bedrock_runtime = get_bedrock_runtime('us-east-1', aws_access_key_id=aws_access_token, aws_secret_access_key=aws_secret_key)
# bedrock_runtime = get_bedrock_runtime('us-east-1')
# llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime, config=config)
llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime)

In [14]:
embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)
# embedding_model = HuggingFaceEmbeddings(model_name=embed_model_name)
# if embedding_model.client.tokenizer.pad_token is None:
    # embedding_model.client.tokenizer.pad_token = embedding_model.client.tokenizer.eos_token

In [15]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embedding_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name='apologetics',
    prefer_grpc=True
)

In [16]:
retriever = qdrant.as_retriever()

In [17]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True)

In [18]:
question = 'What is the ontological argument for the existence of God?'

In [19]:
result_json = qa({'query': question})

<ipython-input-19-621b79b99647>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result_json = qa({'query': question})


In [20]:
result_json

{'query': 'What is the ontological argument for the existence of God?',
 'result': ' The ontological argument for the existence of God is an argument that moves from the concept or idea of God to the existence of God. It comes in two forms: one from the idea of a Perfect Being and the other from the idea of a Necessary Being. The Perfect Being form argues that the mere concept of God as an absolutely perfect being demands that he exist, because existence is a perfection. The Necessary Being form argues that existence is logically necessary to the concept of a Necessary Being, and therefore a Necessary Being (God) necessarily exists.',
 'source_documents': [Document(metadata={'page': 490, 'source': '/content/drive/MyDrive/Apologetics Corpus/Baker Encyclopedia of Christian Apologetics (Baker Reference Library) (Norman L. Geisler).pdf', '_id': '0bc3567d-a92b-4389-8f27-fadbd383837d', '_collection_name': 'apologetics'}, page_content='ontological argument (though not the first to call it thi

In [ ]:
import json

In [ ]:
def convert_langchaindoc_to_dict(doc):
  return {
      'page_content': doc.page_content,
      'metadata': doc.metadata
  }

In [ ]:
result_dict = {
    'query': result_json['query'],
    'answer': result_json['result'],
    'source_documents': [convert_langchaindoc_to_dict(doc) for doc in result_json['source_documents']]
}

In [ ]:
json.dumps(result_dict)

'{"query": "What is the ontological argument for the existence of God?", "answer": " The ontological argument for the existence of God is an argument that moves from the conception of a Perfect or Necessary Being to the existence of such a Being. It was first developed by Anselm and there are two forms of the argument: one from the idea of a Perfect Being and the other from the idea of a Necessary Being. The Perfect Being form of the argument argues that the mere concept of God as an absolutely perfect being demands that he exist, because existence is a perfection and God, being a Perfect Being, cannot lack any perfections. The Necessary Being form of the argument argues that a Necessary Being (God) necessarily exists because existence is logically necessary to the concept of a Necessary Being.", "source_documents": [{"page_content": "ontological argument (though not the first to call it this) was Anselm (1033\\u20131109). In its simplest form it\\nargues from the idea of God to the ex